In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [14]:
params = {
    "command": "python",
    "args": ["search_server.py"],
}

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
    mcp_tools = await server.list_tools()

mcp_tools

[Tool(name='search', title=None, description='Perform a DuckDuckGo web search.\n\n    Args:\n        query: The search query.\n        max_results: Maximum number of results to return.\n        region: Region code (e.g., "in-en", "us-en", "wt-wt" for global).\n    ', inputSchema={'properties': {'query': {'title': 'Query', 'type': 'string'}, 'max_results': {'default': 5, 'title': 'Max Results', 'type': 'integer'}, 'region': {'default': 'wt-wt', 'title': 'Region', 'type': 'string'}}, 'required': ['query'], 'title': 'searchArguments', 'type': 'object'}, outputSchema={'properties': {'result': {'title': 'Result', 'type': 'string'}}, 'required': ['result'], 'title': 'searchOutput', 'type': 'object'}, icons=None, annotations=None, meta=None)]

In [ ]:
# import os
# env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
# params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

# async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as server:
#     mcp_tools = await server.list_tools()

# mcp_tools

[Tool(name='brave_web_search', title=None, description='Performs a web search using the Brave Search API, ideal for general queries, news, articles, and online content. Use this for broad information gathering, recent events, or when you need diverse web sources. Supports pagination, content filtering, and freshness controls. Maximum 20 results per request, with offset for pagination. ', inputSchema={'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'Search query (max 400 chars, 50 words)'}, 'count': {'type': 'number', 'description': 'Number of results (1-20, default 10)', 'default': 10}, 'offset': {'type': 'number', 'description': 'Pagination offset (max 9, default 0)', 'default': 0}}, 'required': ['query']}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='brave_local_search', title=None, description="Searches for local businesses and places using Brave's Local Search API. Best for queries related to physical locations, businesses, re

In [16]:
from openai import AsyncOpenAI
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
google_api_key = os.getenv('GOOGLE_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

In [19]:
from datetime import datetime
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Nvidia stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"

In [ ]:
### Duck Duck Go Search MCP integration

In [ ]:
async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=gemini_model, mcp_servers=[mcp_server])
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Based on the search results, here's a brief summary of Nvidia's outlook:

*   **Focus on AI:** Nvidia is heavily involved in artificial intelligence, driving advances in the field.
*   **Investment in India's Deep Tech:** Nvidia has joined a $2 billion investment alliance to mentor AI startups in India. This indicates a strategic focus on expanding its reach in the AI sector within emerging markets.

Please note that I don't have specific stock price predictions for Nvidia in 2025. For detailed stock analysis and financial forecasts, it's recommended to consult with financial professionals or refer to reputable financial news sources.

[non-fatal] Tracing client error 401: {
  "error": {
    "message": "Incorrect API key provided: AIzaSyC4***************************eAYg. You can find your API key at https://platform.openai.com/account/api-keys.",
    "type": "invalid_request_error",
    "param": null,
    "code": "invalid_api_key"
  }
}


4 Traders and a Researcher

Accounts MCP server    
Fetch (get webpage via a local headless browser)   
Memory.  
DuckDuckGo Search.    
Financial data.  

In [21]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts import Account

load_dotenv(override=True)

True

In [22]:
polygon_api_key = os.getenv("POLYGON_API_KEY")

In [23]:
market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

In [25]:
trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

In [26]:
# brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "uv", "args": ["run", "search_server.py"]}
]

In [27]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers